# Cotação

- Fxs que eu trouxe intactas (sem testar) do antigo projeto web1
- Caso precise no futuro, já tenho um ponto de partida.
- Prefri trazê-las para um NB, assim fica mais fácil testar antes de passar para o projeto.
- Diferente da fx que uso no app atualmente (tvb3), nessas aqui passo um df como argumento (que precisa ser criado antes de chamar a fx). Até porque algumas delas precisam do tipo de ativo para raspar, pois os tipos estarão em lugares diferentes.


### Info Money

In [ ]:
# Algumas poucas urls de tickers não são encontradas. Entre elas GARE11 e ISAE4 que tiveram troca de ticker recentemente.
# Então acredito ser algum problema da base do próprio site.
# Funciona local e na nuvem.

# ---------------------------------------------------------------------------------------------------------------------
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep

def criar_df_cotacao_infomoney(df_mov_financeiras):

    # Obtendo os valores únicos da coluna 'Ativo' e 'Tipo de Ativo' e convertendo em uma lista de listas
    lista_tickers = df_mov_financeiras[['Ativo', 'Tipo de Ativo']].drop_duplicates().values.tolist()

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://www.infomoney.com.br/"
    }

    dados = []

    for ticker, tipo in lista_tickers:

        # É estranho pois a url é assim: https://www.infomoney.com.br/cotacoes/b3/acao/banco-do-brasil-bbas3/
        # Mas eu requisitando a url assim, eu tbm chego no endereço acima: https://www.infomoney.com.br/bbas3/
        url = f"https://www.infomoney.com.br/{ticker.lower()}"

        valor = None

        if url:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")

                # Escolhendo a classe correta com base no tipo de ativo
                if tipo == 'Ação' or tipo == 'ETF':
                    cotacao_elemento = soup.find("div", class_="value")
                    texto_bruto = cotacao_elemento.get_text(strip=True)
                    texto = texto_bruto[0:5]
                    valor = float(texto.replace("R$", "").replace(".", "").replace(",", "."))
                elif tipo == 'FII':
                    cotacao_elemento = soup.find("span", class_="typography__display--2-noscale typography--numeric spacing--mr1")
                    texto = cotacao_elemento.get_text(strip=True)
                    valor = float(texto.replace("R$", "").replace(".", "").replace(",", "."))
                else:
                    cotacao_elemento = None  # Tipo indefinido, não procuramos
                    print(f"Cotação não encontrada para {ticker} na URL: {url}")

            except Exception as e:
                print(f"Erro ao obter {ticker} na URL: {url} | Erro: {e}")

            # Adicionando à lista de dados para o DataFrame
            dados.append({
                "Ticker": ticker,
                "Preço": valor
            })

            sleep(1)  # pausa para evitar bloqueio

    return pd.DataFrame(dados)

# ---------------------------------------------------------------------------------------------------------------------
## Código para investigar formas de obter o arquivo através de NAME e CLASS

# import requests
# import pandas as pd
# from bs4 import BeautifulSoup
# from time import sleep
#
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
#     "Referer": "https://www.infomoney.com.br/"
# }
#
# # É estranho pois a url é assim: https://www.infomoney.com.br/cotacoes/b3/acao/banco-do-brasil-bbas3/
# # Mas eu requisitando a url assim, eu tbm chego no endereço acima: https://www.infomoney.com.br/bbas3/
# url = "https://www.infomoney.com.br/vale3"
#
# response = requests.get(url, headers=headers, timeout=10)
#
# soup = BeautifulSoup(response.text, "html.parser")
#
# # Procurar o elemento que contém a cotação (div com classe "value")
# cotacao_elemento = soup.find("div", class_="value") # classe para fiis="cotacoes__header-price" , ações e etfs "value"
#
# # soup
#
# cotacao_elemento
#
# texto = cotacao_elemento.get_text(strip=True)
#
#
# texto

### Investidor 10

In [ ]:
# Por enquanto funciona local e na nuvem

import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep

def criar_df_cotacao_investidor10(df_mov_financeiras):
    """
    Função para buscar a cotação atual de uma lista de ativos no site Investidor10.
    A URL e classe do html acessada dependem do tipo de ativo: Ação, FII ou ETF.
    Classes para puxar elemento:
        Descobri a classe pelo cod fonte, dando CTRL+f e pesquisando cotação = da página. Lá peguei a classe.
    """

    # Obtendo os valores únicos da coluna 'Ativo' e 'Tipo de Ativo' e convertendo em uma lista de listas
    lista_tickers = df_mov_financeiras[['Ativo', 'Tipo de Ativo']].drop_duplicates().values.tolist()

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://investidor10.com.br"
    }

    dados = []

    for ticker, tipo in lista_tickers:

        # Definindo a URL com base no tipo de ativo
        if tipo == 'Ação':
            url = f"https://investidor10.com.br/acoes/{ticker.lower()}/"
        elif tipo == 'FII':
            url = f"https://investidor10.com.br/fiis/{ticker.lower()}/"
        elif tipo == 'ETF':
            url = f"https://investidor10.com.br/etfs/{ticker.lower()}/"
        else:
            url = None  # Tipo indefinido, pode ser tratado conforme necessário

        valor = None

        if url:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                response.raise_for_status()

                soup = BeautifulSoup(response.text, "html.parser")

                # Escolhendo a classe correta com base no tipo de ativo
                if tipo == 'Ação':
                    cotacao_elemento = soup.find("div", class_="_card-body")
                elif tipo == 'FII':
                    cotacao_elemento = soup.find("div", class_="compare-progress-bar primary")
                elif tipo == 'ETF':
                    cotacao_elemento = soup.find("div", class_="etfCurrentQuotation")
                else:
                    cotacao_elemento = None  # Tipo indefinido, não procuramos

                # Se encontrou o elemento, extrair e converter o valor
                if cotacao_elemento:
                    texto = cotacao_elemento.get_text(strip=True)
                    valor = float(texto.replace("R$", "").replace(".", "").replace(",", "."))
                else:
                    print(f"Cotação não encontrada para {ticker} na URL: {url}")

            except Exception as e:
                print(f"Erro ao obter {ticker} na URL: {url} | Erro: {e}")

        # Adicionando à lista de dados para o DataFrame
        dados.append({
            "Ticker": ticker,
            "Preço": valor
        })

        sleep(1)  # pausa para evitar bloqueio

    return pd.DataFrame(dados)


# ------------------------------------------------------------------------------------------------- manutenção da fx
# Essa estrutura abaixo servirá caso eu queira investigar a cotação de alguma url. Na realidade é a estrura inicial de
# qualquer scrap desses que estou fazendo. Não faria sentido investigar urls com uma fx pronta.
# Preciso editar as urls e outras coisas

# import requests
# import pandas as pd
# from bs4 import BeautifulSoup
# from time import sleep
#
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
#     "Referer": "https://investidor10.com.br"
# }
#
# url = "https://investidor10.com.br/fiis/hglg11"
#
# response = requests.get(url, headers=headers, timeout=10)
#
# soup = BeautifulSoup(response.text, "html.parser")
#
# # Procurar o elemento que contém a cotação (div com classe "value")
# cotacao_elemento = soup.find("div", class_="compare-progress-bar primary")
#
# # soup
#
# cotacao_elemento
#
# texto = cotacao_elemento.get_text(strip=True)
#
# texto




### Money Times

In [ ]:

# -------------------------------------------------------------------------------------------------------------------
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep


def criar_df_cotacao_moneytimes(df_mov_financeiras):

    # Obtendo os valores únicos da coluna 'ATIVO' e convertendo em uma lista
    lista_tickers = df_mov_financeiras['Ativo'].unique().tolist()

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://www.moneytimes.com.br/"
    }

    dados = []

    for ticker in lista_tickers:

        url = f"https://www.moneytimes.com.br/cotacao/{ticker.lower()}/"

        valor = None

        if url:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                response.raise_for_status()

                soup = BeautifulSoup(response.text, "html.parser")

                cotacao_elemento = soup.find("div", class_="valor")

                # Se encontrou o elemento, extrair e converter o valor
                if cotacao_elemento:
                    texto = cotacao_elemento.get_text(strip=True)
                    valor = float(texto.replace("R$", "").replace(".", "").replace(",", "."))
                else:
                    print(f"Cotação não encontrada para {ticker} na URL: {url}")

            except Exception as e:
                print(f"Erro ao obter {ticker} na URL: {url} | Erro: {e}")

                # Adicionando à lista de dados para o DataFrame
            dados.append({
                "Ticker": ticker,
                "Preço": valor
            })

            sleep(1)  # pausa para evitar bloqueio

    return pd.DataFrame(dados)

### Satus Invest

In [ ]:
# # lista_manual_tickers = ['BBAS3', 'TAEE11', 'BRBI11', 'CSMG3', 'PETR4', 'VALE3', 'BBSE3', 'ISAE4', 'HASH11', 'IVVB11',
# #                         'MXRF11', 'HGLG11', 'MXRF11', 'BTLG11', 'GARE11', 'GGRC11', 'RECR11', 'CPTS11', 'HSLG11',
# #                         'XPLG11', 'XPML11', 'GTWR11', 'TAEE3', 'SAPR4', 'ITSA4']
#
# # -------------------------------------------------------------------------------------------------------------------
# Cada tipo de ativo tem uma url diferente
# Funciona no ambiente local, mas dá erro na nuvem
# -------------------------------------------------------------------------------------------------------------------
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep


def criar_df_cotacao_statusinvest(df_mov_financeiras):

    # Obtendo os valores únicos da coluna 'ATIVO' e convertendo em uma lista de tuplas (ticker, tipo)
    lista_tickers = df_mov_financeiras[['Ativo', 'Tipo de Ativo']].drop_duplicates().values.tolist()

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Referer": "https://statusinvest.com.br"
    }

    dados = []

    for ticker, tipo in lista_tickers:

        # Definindo a URL com base no tipo de ativo
        if tipo == 'Ação':
            url = f"https://statusinvest.com.br/acoes/{ticker.lower()}"
        elif tipo == 'FII':
            url = f"https://statusinvest.com.br/fundos-imobiliarios/{ticker.lower()}"
        elif tipo == 'ETF':
            url = f"https://statusinvest.com.br/etfs/{ticker.lower()}"
        else:
            url = None  # Tipo indefinido, pode ser tratado conforme necessário

        valor = None

        if url:
            try:
                response = requests.get(url, headers=headers, timeout=10)
                response.raise_for_status()

                soup = BeautifulSoup(response.text, "html.parser")

                # Procurar o primeiro elemento com classe que contém a cotação
                cotacao_elemento = soup.find("strong", {"class": "value"})

                if cotacao_elemento:
                    texto = cotacao_elemento.get_text(strip=True)
                    valor = float(texto.replace("R$", "").replace(".", "").replace(",", "."))

            except Exception as e:
                print(f"Tentativa falhou para {ticker} na URL: {url} | Erro: {e}")

        # Adicionando à lista de dados para o DataFrame
        dados.append({
            "Ticker": ticker,
            "Preço": valor
        })

        sleep(1)  # pausa para evitar bloqueio

    return pd.DataFrame(dados)


### Yahoo Finance

In [ ]:
# Sempre funcionou localmente. Assim que implementei na nuvem ela funcionou, mas depois de alguns dias testando,
# começou a travar na nuvem e depois travou local também, tanto no projeto web quanto no local.


import yfinance as yf

def criar_df_cotacao_yf(df_mov_financeiras):

    lista_tickers = df_mov_financeiras['Ativo'].unique().tolist()

    # Acrescentando ".SA" a cada elemento da lista e ordenando para que fique na sequência padrão.
    lista_tickers_yf = [ativo + ".SA" for ativo in sorted(lista_tickers)]

    data_api = '2025-05-15' # Se for usar, precisa deixar isso dinâmico.

    df_cotacao_yf = yf.download( # yf.download não precisa de tratamento caso não encontre um ticker, pois ele preenche None e cotinua funcionando.
        lista_tickers_yf,
        start=data_api,
        auto_adjust=False, # Se for cruzar dados c/ div em separado, deixar auto_adjust=False é melhor, senão conta o div duas vezes.
        progress=False, # barra de progresso
    )['Close']  # Escolhe coluna

    return df_cotacao_yf


# --------------------------------------------------------------- Mesma estrutura da fx acima, mas pega 1 cotação por vez
# import yfinance as yf
# import pandas as pd
# import time
#
# def criar_df_cotacao_yf(df_mov_financeiras):
#     lista_tickers = df_mov_financeiras['Ativo'].unique().tolist()
#
#     # Acrescentando ".SA" a cada elemento da lista e ordenando para que fique na sequência padrão.
#     lista_tickers_yf = [ativo + ".SA" for ativo in sorted(lista_tickers)]
#
#     data_api = '2025-05-15'  # Se for usar, precisa deixar isso dinâmico.
#
#     # DataFrame vazio para armazenar os dados
#     df_cotacao_yf = pd.DataFrame()
#
#     for ticker in lista_tickers_yf:
#         try:
#             df_ticker = yf.download(
#                 ticker,
#                 start=data_api,
#                 auto_adjust=False,
#                 progress=False,
#             )['Close']
#
#             df_cotacao_yf[ticker] = df_ticker  # adiciona como nova coluna
#
#         except Exception as e:
#             print(f"Erro ao baixar {ticker}: {e}")
#
#         time.sleep(2)  # pausa de 2 segundos entre os downloads
#
#     return df_cotacao_yf


### Não lembro porque não deixei salva a fx que pega a cotação da fonte "Fundamentos". Provavelmente ela não achava Etfs ou algo assim. No projeto antigo Carteira1 eu devo ter um codigo que puxa a tabela completa.